In [40]:
import pandas as pd
import numpy as np

import time
import datetime

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error

import xgboost as xgb

from sklearn.model_selection._validation import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

Одной из самых простых метрик успешности бизнеса для владельцев кафе и ресторанов является посещаемость их заведения. Это интуитивно понятно - в целом, чем больше клиентов, тем больше прибыль. Соответственно, нужно уметь оценивать этот показатель и понимать, как на него влияют различные параметры, поэтому вам предлагается для решения задача предсказания процента заполненности кафе. В течении дня эта величина может заметно меняться, поэтому приведена информация для обеденного времени в будний день.

В данных представлены как характеристики самих заведений, так и описание окружающих их обьектов.

# Описание файлов

- train.csv - обучающая выборка
- test.csv - тестовая выборка
- answers_sample.csv - формат для посылки
- answers_train.csv - ответы для обучающей выборки

# Описание данных

- lon, lat - координаты по долготе и широте
2. price - средний чек
3. lunch - цена ланча
4. n_seats - количество посадочных мест
5. is_net - индикатор сетевого заведения (1 - сетевое, 0 - уникальное)
6. type - тип заведения - кафе, ресторан и так далее
7. obj_rayon - район расположения
8. sumareaLive, meanareaLive - суммарная и средняя площади (в м2) по жилым обьектам в радиусе 500 метров
9. maxareaLive - площадь самого большого жилого обьекта (в м2) в радиусе 500 метров
8. sumareaComm, meanareaComm, maxareaComm - параметры для коммерческих обьектов, аналогичные жилым
10. places_near_300m - количество ресторанов в радиусе 300 метров
11. metro_near_500m - количество станций метро в радиусе 500 метров
12. dist_to_metro - расстояние до ближайшей станции метро (в метрах)
13. something_300m - количество объектов типа 'something' в радиусе 300 метров (всего 250 признаков)

In [65]:
X_train, X_test = pd.read_csv('train.csv').replace(-1, np.nan).apply(lambda row: row.fillna(np.mean(row)), axis=1), pd.read_csv('test.csv').replace(-1, np.nan).apply(lambda row: row.fillna(np.mean(row)), axis=1)
sample, Y_train = pd.read_csv('answers_sample.csv'), pd.read_csv('answers_train.csv').replace(-1, np.nan).apply(lambda row: row.fillna(np.mean(row)), axis=1)

- dist_to_metro, yes - - 4
- maxareaCom, no
- maxareaLiv, no
- meanareaComm, yes - 4
- meanareaLive, yes - 4
- sumareaCom, no
- sumareaLiv, no

In [66]:
X_test.drop('maxareaComm maxareaLive sumareaComm sumareaLive'.split(), inplace=True, axis=1)
X_train.drop('maxareaComm maxareaLive sumareaComm sumareaLive'.split(), inplace=True, axis=1)
for name in X_train.corr()[lambda x: np.abs(x)>0.7].count()[lambda x: x>8].index:
   X_train.drop(name, inplace=True, axis=1)
   X_test.drop(name, inplace=True, axis=1)

In [67]:
from sklearn.cluster import KMeans
clf = KMeans(n_clusters=3, random_state=1)
for name in 'dist_to_metro meanareaComm meanareaLive'.split():
    Xtr = X_train[name].as_matrix().reshape(-1, 1)
    Xt = X_test[name].as_matrix().reshape(-1, 1)
    clf.fit(Xtr)
    X_train[name] = clf.labels_
    clf.fit(Xt)
    X_test[name] = clf.labels_

In [68]:
from sklearn.linear_model import LogisticRegression

In [71]:
def Testing_grid_reg(X_train, Y_train, c=[10.0 ** i for i in range(-9, 8)]):
    cv = KFold(n_splits=3, shuffle=True, random_state=42)
    grid = {'C': c,
           'penalty' : ['l1', 'l2']}
    clf = LogisticRegression(penalty='l1')
    
    start_time = datetime.datetime.now()
    gs = GridSearchCV(clf, scoring='neg_mean_absolute_error', param_grid=grid, cv=cv, return_train_score=True)
    gs.fit(X_train,Y_train)
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    
    means = gs.cv_results_['mean_test_score']
    stds = gs.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, gs.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    return c[np.argmax(means)], max(means), gs.best_params_

def Testing_grid_xgb(X_train, Y_train):
    cv = KFold(n_splits=3, shuffle=True, random_state=42)
    clf = xgb.XGBRegressor()        
    parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [0.05, .004, 0.002], #so called `eta` value
              'max_depth': [6, 7, 9],
              'min_child_weight': [4, 5, 7],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [1000],
              'base_score' : [0.5]}
    
    start_time = datetime.datetime.now()
    gs = GridSearchCV(clf, scoring='neg_mean_absolute_error', param_grid=parameters, cv=cv,  n_jobs = 5, verbose=True)
    gs.fit(X_train,Y_train)
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    
    means = gs.cv_results_['mean_test_score']
    stds = gs.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, gs.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    return max(means), gs.best_params_

In [72]:
def Res_save(X_train, Y_train, X_test,clf):
    clf.fit(X_train.as_matrix(), Y_train['answer'].as_matrix())
    rez = clf.predict(X_test.as_matrix())
    Results = pd.concat([pd.DataFrame(X_test.index), pd.DataFrame(rez)], axis=1)
    Results.columns = ['id', 'answer']
    Results.to_csv('result.csv', index=False)

In [73]:
params = Testing_grid_xgb(X_train.as_matrix(), Y_train['answer'].as_matrix())[1]

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  1.3min
[Parallel(n_jobs=5)]: Done  81 out of  81 | elapsed:  2.4min finished


Time elapsed: 0:02:27.062887
-17.830 (+/-1.878) for {'base_score': 0.5, 'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 4, 'n_estimators': 1000, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}
-17.710 (+/-2.143) for {'base_score': 0.5, 'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 5, 'n_estimators': 1000, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}
-17.732 (+/-1.704) for {'base_score': 0.5, 'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 7, 'n_estimators': 1000, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}
-17.764 (+/-2.081) for {'base_score': 0.5, 'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 1000, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}
-17.494 (+/-2.086) for {'base_score': 0.5, 'colsample_bytree': 0.7, 'learni

In [75]:
for key in params:
    print (key,'=',params[key],',')

base_score = 0.5 ,
colsample_bytree = 0.7 ,
learning_rate = 0.002 ,
max_depth = 9 ,
min_child_weight = 4 ,
n_estimators = 1000 ,
nthread = 4 ,
objective = reg:linear ,
silent = 1 ,
subsample = 0.7 ,


In [76]:
clf = xgb.XGBRegressor(base_score = 0.5, colsample_bytree = 0.7 ,
learning_rate = 0.0002 ,
max_depth = 9 ,
min_child_weight = 4 ,
n_estimators = 1000 ,
nthread = 4 ,
objective = 'reg:linear' ,
silent = 1 ,
subsample = 0.7)

In [37]:
Testing_grid_reg(X_train.as_matrix(), Y_train['answer'].as_matrix())

Time elapsed: 0:00:37.039158
-25.380 (+/-0.903) for {'C': 1e-09, 'penalty': 'l1'}
-20.067 (+/-1.239) for {'C': 1e-09, 'penalty': 'l2'}
-25.380 (+/-0.903) for {'C': 1e-08, 'penalty': 'l1'}
-20.058 (+/-0.922) for {'C': 1e-08, 'penalty': 'l2'}
-25.380 (+/-0.903) for {'C': 1e-07, 'penalty': 'l1'}
-20.048 (+/-0.856) for {'C': 1e-07, 'penalty': 'l2'}
-25.380 (+/-0.903) for {'C': 1e-06, 'penalty': 'l1'}
-20.035 (+/-0.820) for {'C': 1e-06, 'penalty': 'l2'}
-19.473 (+/-2.451) for {'C': 1e-05, 'penalty': 'l1'}
-20.230 (+/-1.132) for {'C': 1e-05, 'penalty': 'l2'}
-19.974 (+/-0.935) for {'C': 0.0001, 'penalty': 'l1'}
-20.048 (+/-1.299) for {'C': 0.0001, 'penalty': 'l2'}
-20.042 (+/-0.799) for {'C': 0.001, 'penalty': 'l1'}
-18.332 (+/-0.568) for {'C': 0.001, 'penalty': 'l2'}
-19.399 (+/-1.481) for {'C': 0.01, 'penalty': 'l1'}
-18.847 (+/-1.602) for {'C': 0.01, 'penalty': 'l2'}
-19.217 (+/-2.718) for {'C': 0.1, 'penalty': 'l1'}
-20.272 (+/-2.702) for {'C': 0.1, 'penalty': 'l2'}
-21.198 (+/-1.629) fo

(10000.0, -18.332268370607029, {'C': 0.001, 'penalty': 'l2'})

In [38]:
clf = LogisticRegression(C=0.001, penalty= 'l2')

In [77]:
Res_save(X_train, Y_train, X_test, clf=clf)

In [27]:
params

{'base_score': 0.1,
 'colsample_bytree': 0.7,
 'learning_rate': 0.004,
 'max_depth': 7,
 'min_child_weight': 4,
 'n_estimators': 500,
 'nthread': 4,
 'objective': 'reg:linear',
 'silent': 1,
 'subsample': 0.7}

In [23]:
X_train.corr()[lambda x: np.abs(x)>0.6].count()[lambda x: x>3].index

Index(['PR-агентство_300m', 'Агентство недвижимости_300m',
       'Брокерская компания_300m', 'Железнодорожные и авиабилеты_300m',
       'Жильё посуточно_300m', 'Кадровое агентство_300m', 'Кальян-бар_300m',
       'Квесты_300m', 'Клуб досуга_300m', 'Косметология_300m',
       'Магазин одежды_300m', 'Магазин чая и кофе_300m',
       'Музыкальное образование_300m', 'Ногтевая студия_300m',
       'Общественная организация_300m', 'Оценочная компания_300m',
       'Помощь в оформлении виз и загранпаспортов_300m', 'Пункт выдачи_300m',
       'Спортивное объединение_300m', 'Строительные и отделочные работы_300m',
       'Студия веб-дизайна_300m', 'Товары для мобильных телефонов_300m',
       'Турагентство_300m', 'Туроператор_300m', 'Учебный центр_300m',
       'Художественная мастерская_300m', 'Центр йоги_300m',
       'Школа танцев_300m', 'Юридические услуги_300m'],
      dtype='object')